In [69]:
import pandas as pd 
import os   
from glob import glob
#library to read text from pdfs
import fitz

In [70]:
#gives us the current working directory
curr_directory = os.getcwd()
#print(curr_directory)
#gives the directory where all pdfs are stored
profile_path = os.path.join(curr_directory + "\\profile_pdfs\\")
#print(profile_path)
list_pdf_profile_paths = glob(os.path.join(profile_path,"*.{}".format('pdf')))
#print(list_pdf_profile_paths[:5])

TASK 1 Complete and Task 2 start

In [71]:
#reading text from pdf files
def extractTextFromPDF(filepath):
    doc = fitz.open(filepath)                      # open document
    #doc object of fitz stores the document page objects
    all_text = ""
    for page in doc:
        #print(page.getText()) 
        #page.getText() returns the text contained in a single page
        all_text = all_text  + page.getText()

    return all_text
    #print(all_text)


In [72]:
#fetching text from all profiles in a list
text_all_profiles = [extractTextFromPDF(path) for path in list_pdf_profile_paths]


In [73]:
df_profiles = pd.DataFrame()
df_profiles['SNo'] = [i for i in range(1,51)]
df_profiles['Profile_Text'] = text_all_profiles
df_profiles.head()
df_profiles.to_csv("Output\\submission_Task2.csv",index=False)

TASK 2 COMPLETE

In [74]:
from nltk import pos_tag
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import operator
import re

In [75]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        #print(wordnet.ADJ)
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV

In [76]:
def cleanAndTokenizeText(text):
    tokenizer = RegexpTokenizer(r'\w+')
    en_stop = stopwords.words('english')
    #this extendes the list of stopwords 
    en_stop.extend(['www','gmail','month','page','linkedin','com','india','year'])
    months = ['january','february','march','april','may','june','july','august','september','october','november','december']
    en_stop.extend(months)
    en_stop = set(en_stop)    
    text = text.lower()
    tokens = tokenizer.tokenize(text)
    stopped_tokens = [j for j in tokens if not j in en_stop]
    stopped_tokens = [j for j in stopped_tokens if len(j)>1]
    pos_tokens = pos_tag(stopped_tokens)
    tokens_final = []
    wordnet_lemmatizer = WordNetLemmatizer()

    for word in pos_tokens:
        try:
            lem_word = wordnet_lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
            #print ("{0:20}{1:20}".format(word[0],lem_word))
            tokens_final.append(lem_word)
        except:
            #tokens_final.append(word[0])    #uncomment if you want to include numbers and unknown words
            pass
            #to handle errors where words are digits or words which are not known in English Language or other errors

    tokens_final = [j for j in tokens_final if not j in en_stop]
    return tokens_final


In [77]:
def getMostFrequentWords(text):
    list_words = cleanAndTokenizeText(text)
    freq_words = {}
    for word in list_words:
        if(word not in freq_words):
            freq_words[word] = 1
        else:
            freq_words[word]+=1 

    sorted_freq_words =  sorted(freq_words.items(), key=operator.itemgetter(1),reverse=True)
    most_frequent_words = [i[0] for i in sorted_freq_words]
    most_frequent_words = most_frequent_words[0:20]
    return most_frequent_words
    #print("Top 20 words freuency wise are:",sorted_freq_words[:20])

In [78]:
df_profiles['Most_Frequent_Words'] = df_profiles['Profile_Text'].apply(lambda x:getMostFrequentWords(x))
df_profiles.to_csv("Output\\submission_Task3_mostFreqWords.csv",index=False)

In [79]:
import math
from textblob import TextBlob as tb

def tf(word, blob):
    return blob.words.count(word) / len(blob.words)

def n_containing(word, bloblist):
    return sum(1 for blob in bloblist if word in blob.words)

def idf(word, bloblist):
    return math.log(len(bloblist) / (1 + n_containing(word, bloblist)))

def tfidf(word, blob, bloblist):
    return tf(word, blob) * idf(word, bloblist)

In [80]:
def getMostImpWords(list_of_documents_text):
    
    bloblist = [cleanAndTokenizeText(blob) for blob in list_of_documents_text]
    bloblist = [" ".join(i for i in blob) for blob in bloblist]
    bloblist = [tb(blob) for blob in bloblist]
    essential_words = []
    for i, blob in enumerate(bloblist):
        #print("Top words in document {}".format(i + 1))
        scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
        sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        imp_words = []
        for word, score in sorted_words[:10]:
            imp_words.append(word)
            #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
        essential_words.append(imp_words)

    return essential_words

In [82]:
list_of_documents_text = list(df_profiles['Profile_Text'])
df_profiles['Most Important Words'] = getMostImpWords(list_of_documents_text)
df_profiles.to_csv("Output\Submission_Task3_MostImpWords.csv",index = False)

# Rough work ,understanding, practice

text = text_test

Tokenizing and removing special characters

#\w+ captures characters,numbers and underscore and removes all special charcters as well as spaces
tokenizer = RegexpTokenizer(r'\w+')
#create English stop words list
en_stop = stopwords.words('english')
#this extendes the list of stopwords 
en_stop.extend(['www','gmail','month','page','linkedin','com','india'])
months = ['january','february','march','april','may','june','july','august','september','october','november','december']
en_stop.extend(months)
#this makes the list unique
en_stop = set(en_stop)    
text = text.lower()
#this splits the string into tokens or words
tokens = tokenizer.tokenize(text)
#print(tokens)
print(len(tokens))


Removing stop Words

# remove stop words from tokens
stopped_tokens = [j for j in tokens if not j in en_stop]
#remove one character words from tokens    
stopped_tokens = [j for j in stopped_tokens if len(j)>1]
#print(stopped_tokens)
print(len(stopped_tokens))


Bringing the words to its root form based on parts of speech tagging and lemmatization

pos_tokens = pos_tag(stopped_tokens)
print(pos_tokens[0:20])


tokens_final = []
wordnet_lemmatizer = WordNetLemmatizer()

for word in pos_tokens:
    try:
        lem_word = wordnet_lemmatizer.lemmatize(word[0], pos=get_wordnet_pos(word[1]))
        #print ("{0:20}{1:20}".format(word[0],lem_word))
        tokens_final.append(lem_word)
    except:
        #tokens_final.append(word[0])
        pass
        #print("\n!!Error here:",word[0],"!!\n") 
        #to handle errors where words are digits or words which are not known in English Language



#just to confirm removal if any new stop_words were introduced are lemmatization
tokens_final = [j for j in tokens_final if not j in en_stop]
len(tokens_final)

freq_words = {}
for word in tokens_final:
    if(word not in freq_words):
        freq_words[word] = 1
    else:
        freq_words[word]+=1 

sorted_freq_words =  sorted(freq_words.items(), key=operator.itemgetter(1),reverse=True)
print("Top 20 words frequency wise are:",sorted_freq_words[:20])

bloblist = list(df_profiles['Profile_Text'])
bloblist = [cleanAndTokenizeText(blob) for blob in bloblist ]
bloblist = [" ".join(i for i in blob) for blob in bloblist]
bloblist = [tb(blob) for blob in bloblist]
#bloblist[0]

bloblist = text.split("\n")
bloblist = [cleanAndTokenizeText(blob) for blob in bloblist ]
bloblist = [" ".join(i for i in blob) for blob in bloblist]
bloblist = [blob for blob in bloblist if blob!='']
bloblist = [tb(blob) for blob in bloblist]
print(bloblist)

essential_words = []
for i, blob in enumerate(bloblist):
    #print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    sorted_words = sorted(scores.items(), key=lambda x: x[1], reverse=True)
    imp_words = []
    for word, score in sorted_words[:10]:
        imp_words.append(word)
        #print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))
    essential_words.append(imp_words)

scores_all = {}

for i, blob in enumerate(bloblist):
    #print("Top words in document {}".format(i + 1))
    scores = {word: tfidf(word, blob, bloblist) for word in blob.words}
    scores_all.update(scores)

sorted_words = sorted(scores_all.items(), key=lambda x: x[1], reverse=True)
for word, score in sorted_words[:10]:
    print("\tWord: {}, TF-IDF: {}".format(word, round(score, 5)))